In [61]:
# Display the dataframe
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

In [69]:
import pandas as pd
import datetime

In [33]:
df = pd.read_csv("data/sample.csv", index_col="Unnamed: 0")
crowd_sample = df.iloc[:, 0:4]

crowd_sample["Datetime"] = pd.to_datetime(crowd_sample['Time'])
crowd_sample.set_index(["Datetime"], inplace=True)
crowd_sample = crowd_sample.drop(["Time"], axis=1)

In [34]:
crowd_sample.head()

,CMSA-GAWW-22 Kloveniersburgwal,CMSA-GAWW-23 Bloedstraat,CMSA-GAWW-14 Oudezijds Voorburgwal t.h.v. 91
Datetime,,,
2021-04-10 00:00:00,57,73,77
2021-04-10 00:15:00,57,42,66
2021-04-10 00:30:00,43,42,73
2021-04-10 00:45:00,33,27,71
2021-04-10 01:00:00,38,31,72


In [35]:
crowd_sample.index

DatetimeIndex(['2021-04-10 00:00:00', '2021-04-10 00:15:00',
               '2021-04-10 00:30:00', '2021-04-10 00:45:00',
               '2021-04-10 01:00:00', '2021-04-10 01:15:00',
               '2021-04-10 01:30:00', '2021-04-10 01:45:00',
               '2021-04-10 02:00:00', '2021-04-10 02:15:00',
               ...
               '2021-10-10 21:30:00', '2021-10-10 21:45:00',
               '2021-10-10 22:00:00', '2021-10-10 22:15:00',
               '2021-10-10 22:30:00', '2021-10-10 22:45:00',
               '2021-10-10 23:00:00', '2021-10-10 23:15:00',
               '2021-10-10 23:30:00', '2021-10-10 23:45:00'],
              dtype='datetime64[ns]', name='Datetime', length=672, freq=None)

In [36]:
# Any null values
missing_val_count_by_column = (crowd_sample.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

Series([], dtype: int64)


In [38]:
crowd_sample.size

2016

In [95]:
sensor1 = pd.read_csv("data/CMSA-GAWW-22-Kloveniersburgwal.csv")
sensor1["Datetime"] = pd.to_datetime(sensor1['Datetime'])
# df.rename(columns={"A": "a", "B": "c"})
sensor1 = sensor1.rename(columns={"CMSA-GAWW-22 Kloveniersburgwal": "count"})

In [96]:
sensor1.head()

,Datetime,count
0,2021-04-10 00:00:00,57
1,2021-04-10 00:15:00,57
2,2021-04-10 00:30:00,43
3,2021-04-10 00:45:00,33
4,2021-04-10 01:00:00,38


In [184]:
# Crowd prediction based on simple ARMA model, using sample data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA
import warnings

# Specify to ignore warning messages
warnings.filterwarnings("ignore")

# Training set
train = sensor1[-100:]
arima_fit = ARIMA(train["count"].values, order=(1,0,1)).fit()
fc, se, conf  = arima_fit.forecast(100, alpha=0.05)  # 95% conf

dt = [train.iloc[-1:]["Datetime"].values[0] + np.timedelta64(15*i,'m') for i in range(100)]
forecasts = pd.DataFrame({"datetime": dt})
forecasts["forecast"] = fc 
forecasts["lower_bound"] = conf[:, 0]
forecasts["upper_bound"] = conf[:, 1]
# lower_bound = pd.Series(conf[:, 0], index=index)
# upper_bound = pd.Series(conf[:, 1], index=index)

In [187]:
forecasts.to_csv("result/sensor1_forecast.csv")